In [ ]:
# Ignorar sklearn warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='skimage')

# Librería de ámbito matemático, científico y de ingeniería. En nuestro caso, solo usamos la convolución.
import scipy.io

# Interfaz para HDF5. Solo necesario para cargar las anotaciones del dataset.
import hdf5storage

# Librería para la computación científica. En nuestro caso, la usamos por la representación de datos y hacer calculas de forma
# rápida.
import numpy as np

# Librerías de visión por computador
# OpenCV: Generalista.
# Skimage: Generalista.
# Dlib: Menos funcionalidades, muy rápido e implementaciones muy interesantes.
import dlib
import cv2
from skimage import io
from skimage import color
from skimage import filters
from skimage import util
from skimage import transform
from skimage import feature

# Librerías para gráficos en 2d/3d. En nuestro caso para mostrar las imágenes y gráficos en general.
from matplotlib import pyplot as plt

# Necesario para visualizar los resultados en la misma página.
%matplotlib inline

Para comprender mejor las explicaciones de la presentación procederemos a entrenar un detector paso por paso. Recomiendo entender lo que está pasando, pero intentar no perder tiempo en detalles insignificantes (si no es que os motiva mucho!). La mayoría de detectores se entrenan usando procesos end2end donde no hay que hacer más que alimentar una implementación ya hecha con datos de entrada, y/o tunear detectores ya existentes para resolver nuestro caso concreto.

A continuación vamos a programar nuestro primer detector como profundización de los conceptos teóricos, y en las siguientes secciones veremos cómo entrenar nuestro detector de forma semiautomática.

# Entrenando un detector de caras de forma manual

## Obtener un set de muestras positivas y negativas

Para obtener nuestro set usaremos una base de datos que ya tiene "anotadas" las caras y que encontramos disponible en nuestra carpeta de recursos.

Ruta:
__resources\face_dataset__.

In [ ]:
# Cargar las anotaciones del dataset.
anotaciones_path = "resources/session2/face_dataset/anno.mat"
anotaciones = hdf5storage.loadmat(anotaciones_path)
anotaciones = anotaciones["anno"]

In [ ]:
# Estructura de las anotaciones. Esta varia mucha según que de donde proviene el dataset.
print("Dimensiones: ", anotaciones.shape)
print("\nEjemplo de datos de una imagen:")
print("-"*55)
print("Nombre del fichero: ", anotaciones[0][0][0])
print("Bboxes:\n %s" % anotaciones[0][1][0][0])
print("Posición de la cara [X axis, Y axis, Z axis]:\n %s" % anotaciones[0][2][0][0])
print("Landmarks:\n %s" % anotaciones[0][3][0][0])

In [ ]:
# Numero de muestras
print("Numero de imagenes:", len(anotaciones))
print("Numero de caras anotadas:", np.sum([len(anotacion[1][0]) for anotacion in anotaciones]))

In [ ]:
import random # Generar números aleatorios

# Seleccionar una imagen de forma aleatoria
random_image = random.randint(0, (len(anotaciones)))
filename = anotaciones[random_image][0][0][0]
bboxs = anotaciones[random_image][1][0]
landing_marks = anotaciones[random_image][3][0]

# Estructura
f, (ax0, ax1) = plt.subplots(1, 2, figsize=(20, 10))

# Abrir la imágen
img = io.imread("resources/session2/face_dataset/" + filename)
ax0.imshow(img);
ax0.set_title("Imagen sin anotar")

# Dibujar la información disponible
for bbox, bbox_landing_marks in zip(bboxs, landing_marks):
    # Rectangle
    (x1, y1), (x2, y2) = bbox.astype(int)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
    # Landmarks
    for landmark in bbox_landing_marks:
        x, y = landmark.astype(int)
        cv2.circle(img, (x, y) , 7, (255, 0, 0), -1)
        
# Mostrar la imagen
ax1.imshow(img);
ax1.set_title("Imagen anotada");

### Muestras positivas
Ahora que ya conocemos un poco mejor la estructura de datos de las anotaciones, procedemos a "recortar" solo los fragmentos de imagen que representan caras.

In [ ]:
num_samples_positivos = 0
tamaño_redimensionar = (75, 80)

for imagen in anotaciones:
    
    # Obtener información
    nombre_imagen = imagen[0][0][0]
    bboxs = imagen[1][0]
    
    # Cargar imagen
    path = "resources/session2/face_dataset/" + nombre_imagen
    img = io.imread(path)
    # Mensaje debug
    print("Imagen actual:", path)
    print("Número de bbox:", len(bboxs))
    
    for bbox in bboxs:
        # Seleccionar los puntos
        (x1, y1), (x2, y2) = bbox.astype(int)
        # Recortar la imagen
        recorte = img[y1:y2, x1:x2, :]
        # Redimensionar todas al mismo tamaño
        recorte = transform.resize(recorte, tamaño_redimensionar, mode='constant')
        # Guardar el recorte
        filename = "resources/session2/our_face_dataset/positive_samples/%s.jpg" % (str(num_samples_positivos)) 
        io.imsave(filename, recorte)
        
        num_samples_positivos +=1

### Muestras negativas
Ahora lo mismo para la muestras negativas, nuestro objetivo es conseguir recortes de cosas que no sean caras.

In [ ]:
# Cogemos 4 veces el numero de samples positivos
num_samples_negativos = num_samples_positivos * 4
num_imagenes = len(anotaciones)
muestras_por_imagen = int(np.ceil(num_samples_negativos / num_imagenes))

alto_recorte, ancho_recorte = tamaño_redimensionar
print("Alto recorte: %d, Ancho recorte: %d" % (alto_recorte, ancho_recorte))

for imagen in anotaciones:

    # Cargar imagen
    nombre_imagen = imagen[0][0][0]
    path = "resources/session2/face_dataset/" + nombre_imagen
    img = io.imread(path)
    
    print("Imagen actual: %s, dimensiones: %s" % (path, str(img.shape)))
    alto, ancho, canales = img.shape
        
    # Extraer muestras
    for num_muestra in range(muestras_por_imagen):
          
        # Punto aleatorio dentro de la imagen
        y1 = random.randint(0, img.shape[0])
        x1 = random.randint(0, img.shape[1])
        
        while ((y1 + alto_recorte >= alto) 
               or (x1 + ancho_recorte >= ancho)):
            x1 = random.randint(0, (img.shape[0]))
            y1 = random.randint(0, (img.shape[1]))
            
        x2, y2 = x1 + ancho_recorte, y1 + alto_recorte
        
        # Recortamos
        recorte = img[y1:y2, x1:x2, :]
        print("\tRecorte aleatorio: (%d, %d), (%d, %d)" % (x1, y1, x2, y2))

        # Guardamos el resultado
        filename = "resources/session2/our_face_dataset/negative_samples/%s.jpg" % str(num_samples_negativos)
        io.imsave(filename, recorte)
        
        num_samples_negativos -= 1

__Anotaciones__
* ¿Cuál es la imagen con más bbox?
* Hay un error al generar las muestras negativas, ¿podrías decir cuál es?

## Calcular el descriptor Histogram of Gradient para todas las muestras

In [ ]:
import glob # Listado de directorios

num_samples_negativos = muestras_por_imagen * num_imagenes
total_samples = num_samples_positivos + num_samples_negativos

print("Muestras positivas: %d, Muestras negativas: %d , Total muestras: %d" % 
      (num_samples_positivos, num_samples_negativos, total_samples))

tamaño_descriptor = 2592
descriptores = np.zeros((total_samples, tamaño_descriptor))

actual_sample = 0
## Calcular HOG para todas las muestras positivas
nombre_carpeta = "resources/session2/our_face_dataset/positive_samples/"
for filename in glob.glob(nombre_carpeta + '*.jpg'):
    # Cargar muestra
    image = io.imread(filename,  as_grey=True)
    
    if actual_sample % 200 == 0:
        print("Calculando descriptor de la imagen %d/%d" % (actual_sample, total_samples))

    # Calcular descriptor HOG
    descriptor = feature.hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                             transform_sqrt=True, visualise=False, block_norm="L2-Hys")
    descriptores[actual_sample, :] = descriptor
    actual_sample += 1

## Calcular HOG para todas las muestras negativas
nombre_carpeta = "resources/session2/our_face_dataset/negative_samples/"
for filename in glob.glob(nombre_carpeta + '*.jpg'):
    image = io.imread(filename,  as_grey=True)
    
    if actual_sample % 200 == 0:
        print("Calculando descriptor de la imagen %d/%d" % (actual_sample, total_samples))
    
    descriptor = feature.hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                             transform_sqrt=True, visualise=False, block_norm="L2-Hys")
    descriptores[actual_sample, :] = descriptor
    actual_sample += 1
print("Calculando descriptor de la imagen %d/%d" % (actual_sample, total_samples))

# Etiquetas
labels = np.zeros((descriptores.shape[0]))
labels[0:num_samples_positivos] = 1

__Preguntas__
* De que depende el tamaño del descriptor HoG?
* Que significan los parametros de la función feature.hog()?

## Normalizar los datos
Escalamos los datos usando la Clase StandardScaler y dividimos nuestros datos en test, train.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

print("Dimensiones descriptor: %s" % (str(descriptores.shape)))
print("Dimensiones labels: %s" % (str(labels.shape)))

# Normalizamos los valores
print("Ejemplo antes de normalizar: %s" % (str(descriptores[200,:])))
scaler = StandardScaler().fit(descriptores)
descriptors_scaled = scaler.transform(descriptores)
print("Ejemplo después de normalizar: %s" % (str(descriptors_scaled[200,:])))

# Dividimos el dataset en dos partes entrenamiento/test.
X_train, X_test, y_train, y_test = train_test_split(
    descriptors_scaled, labels, test_size=0.1, random_state=0)

## Entrenar el clasificador binario
Ahora entrenaremos el clasificador SVM (support vector machine) que he explicado en los slides, para ello usaremos basicamente la clase SVC que implementa el clasificador SVM, y luego la clase GridSearchCv que nos permitira optimizar los parametros de nuestro clasificador.

In [ ]:
# Esto es un CTRL-C + CTRL-V de los tutorial de Sklearn: 
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}]#,
                    #{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score, n_jobs=4)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()


__Preguntas__
* ¿Qué parámetros son los buenos?
* ¿Qué significa recall y precision?

## Primera prueba del clasificador de caras
A continuación una pequeña prueba para ver si tiene más o menos sentido lo que hemos hecho hasta ahora.

In [ ]:
# Generar una lista de imágenes detro de un directorio
nombre_carpeta = "resources/session2/our_face_dataset/test/"
lista_imagenes =glob.glob(nombre_carpeta + '*.jpg')

# Generamos una figura con múltiples imágenes
fig=plt.figure(figsize=(2, 5))
fig.set_size_inches(18.5, 10.5, forward=True)

for i in range(1, len(lista_imagenes) + 1):
    
    # Cargar imagen en grayscale y reescalar
    image = io.imread(lista_imagenes[i-1], as_grey=True)
    image = transform.resize(image, tamaño_redimensionar, mode='constant')
    
    # Mostramos la imagen.
    ax1 = fig.add_subplot(2, 5, i)
    plt.imshow(image, cmap="gray")
    
    # Calculamos el descriptor lo normalizamos.
    descriptor = feature.hog(image, orientations=9, pixels_per_cell=(8, 8),  cells_per_block=(2, 2),
                             transform_sqrt=True, visualise=False, block_norm="L2-Hys")
    descriptor_scaled = scaler.transform(descriptor.reshape(1, -1))
    
    # Usando nuestro clasificador inferimos que tipo de imagen se trata.
    if clf.predict(descriptor_scaled) == 1:
        ax1.set_title("Cara")
    else:
        ax1.set_title("Otro")

## Sliding windows y pirámide.
Ahora sabemos que nuestro clasificador funciona, pero, ¿cómo hacemos para detectar las caras en una imagen corriente? A continuación se detallan los tres pasos necesarios para ello:

* Generar una pirámide de imágenes con diferentes dimensiones.
* Buscar caras en la imagen actual.
* Eliminar los bbox que se solapan.

### Generar la pirámide 

In [ ]:
image = io.imread("resources/session2/our_face_dataset/test/test.png", as_grey=True)

# Calculamos la pirámide.
pyramid = tuple(transform.pyramid_gaussian(image, max_layer=6, downscale=1.2))

# Concatenar diferentes niveles para entender un poco mas. (No hace falta enteder)
nlevels = 6
row, col = image.shape
montaje = np.zeros((row, col * nlevels))
offset = 0
for i in range(nlevels):
    montaje[:pyramid[i].shape[0], (i * col) - offset:i * col - offset + pyramid[i].shape[1]] = pyramid[i]
    offset += col - pyramid[i].shape[1]

# Mostrar la pirámide.
fig = plt.figure()
fig.set_size_inches(18.5, 20.5, forward=True)
plt.imshow(montaje, cmap="gray");


__Preguntas__
* ¿Por qué tenemos que usar diferentes dimensiones de imagen?
* ¿Cuantas veces reducimos la imagen?
* ¿Para qué nos podría servir incrementar el tamaño de la imagen?

### Sliding windows

In [ ]:
# Generar las ventanas mòviles para el primer nivel de la pirámide.
images = util.view_as_windows(pyramid[0], tamaño_redimensionar, step=16)
print("Dimensiones de el objeto: %s." % str(images.shape))
# Convertir en un array de imágenes
images = images.reshape(images.shape[0] * images.shape[1], images.shape[2], images.shape[3])
print("Dimensiones de el objeto: %s." % str(images.shape))

# Generamos una figura con múltiples imágenes
numero_de_slices = 6
fig=plt.figure(figsize=(numero_de_slices, numero_de_slices))
fig.set_size_inches(18.5, 20.5, forward=True)

for i in range(1, (numero_de_slices * numero_de_slices) + 1):
    
    # Cargar imagen en grayscale y reescalar
    image = images[i-1]
    
    # Mostrar la imagen
    ax1 = fig.add_subplot(6, 6, i)
    plt.imshow(image, cmap="gray")
    
    # Calculamos el descriptor lo normalizamos
    descriptor = feature.hog(image, orientations=9, pixels_per_cell=(8, 8),  cells_per_block=(2, 2),
                             transform_sqrt=True, visualise=False, block_norm="L2-Hys")
    descriptor_scaled = scaler.transform(descriptor.reshape(1, -1))
    
    # Usando nuestro clasificado inferimos que tipo de imagen es
    if clf.predict(descriptor_scaled) == 1:
        ax1.set_title("Cara")
    else:
        ax1.set_title("Otro")

__Preguntas__
* ¿Que define el parámetro step en la función util.view_as_windows?

# Todo junto

In [ ]:
# Parameters
step = 5 # 5-20
downscale = 1.15 # 1.05-1.15
max_layers = 4 # 3-15

image = io.imread("resources/session2/our_face_dataset/test/test.png", as_grey=True)
pyramid = tuple(transform.pyramid_gaussian(image, max_layer=max_layers, downscale=downscale))

# Level 0 - Imagen original.
final_image = io.imread("resources/session2/our_face_dataset/test/test.png")
slices = util.view_as_windows(pyramid[0], tamaño_redimensionar, step=step)
print("Numero de slices: %d" % (slices.shape[0] * slices.shape[1]))
bboxes = []
for row in range(slices.shape[0]):
    for col in range(slices.shape[1]):
        
        #Posiciones
        slice_row_start= row * step
        slice_col_start = col * step
        slice_row_end= slice_row_start  + tamaño_redimensionar[0]
        slice_col_end = slice_col_start  + tamaño_redimensionar[1]
        lt_point = (slice_col_start, slice_row_start)
        rb_point = (slice_col_end, slice_row_end)
        bbox = (slice_col_start, slice_row_start, slice_col_end, slice_row_end)
        
        # Calcular el descriptor para la imagen.
        descriptor = feature.hog(slices[row, col,:, :], orientations=9, pixels_per_cell=(8, 8),  cells_per_block=(2, 2),
                             transform_sqrt=True, visualise=False, block_norm="L2-Hys")
        # Normalizarlo.
        descriptor_scaled = scaler.transform(descriptor.reshape(1, -1))
        
        # Inferéncia
        if clf.predict(descriptor_scaled) == 1:
            cv2.rectangle(final_image, lt_point, rb_point, (0, 255, 0), 3)
            bboxes.append(bbox)

bboxes = np.array(bboxes)
plt.figure(figsize=(50, 50))
plt.imshow(final_image);

### Eliminar los bbox que se solapan.

In [ ]:
# From: https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/, con comentarios traducidos.

def non_max_suppression_fast(boxes, overlapThresh):
    # Si no hay bbox devuelve una lista vacía
    if len(boxes) == 0:
        return []
 
    # Como vamos a calcular la solapación entre bbox nos interesa definir las
    # posiciones con numeros flotantes para no perder precisión.
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    # Inicializar la lista de bbox seleccionados
    pick = []
 
    # Inicializar arrays con las coordenadas de todos los bbox
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
    # Calcular el area de todos los bbox y ordenarlos según el punto inferior derecho.
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
 
    # Iterar mientras queden elementos en la idx
    while len(idxs) > 0:
        # Cojer el último elemento de la lista y añadirlo a la selección
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        # Buscar las coordenadas mas grandes del punto superior izquierdo
        # y lo mismo para el punto inferior derecho.
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
 
        # Calcular el ancho y alto del bbox.
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
 
        # compute the ratio of overlap.
        overlap = (w * h) / area[idxs[:last]]
 
        # Eliminar del idx array todos los elementos que se sobrepasan el 
        # threshold especificado.
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
 
    # Devolver los seleccionados volviendo a convertir las coordenadas en enteros.
    return boxes[pick].astype("int")

In [ ]:
print("Boxes antes de filtrar: %d" % len(bboxes))
bboxes_nm = non_max_suppression_fast(bboxes,0.05)
print("Boxes antes de filtrar: %d" % len(bboxes_nm))

# Level 0 - Imagen original
final_image = io.imread("resources/session2/our_face_dataset/test/test.png")
for bbox in bboxes_nm:
    cv2.rectangle(final_image,(bbox[0],bbox[1]),(bbox[2], bbox[3]),(0,255,0),3)
        
plt.figure(figsize=(50, 50))
plt.imshow(final_image);

# Entrenando un detector de caras de forma automática

Debido a los recursos que tenemos en el momento de la presentación solo és possible entrenar un clasificador "clasico" y no uno con redes neuronales todo y que el metódo para hacerlo con esta libreria por ejemplo es muy similar a lo que haremos a continuación.

## Definir nuestra clase de interés
En la librería Dlib hay varias maneras de definir los bbox, os muestro esta por que es la mas senzilla y que en un caso real seria facil de usar. Pero en este proceso se puede ser creativo para ahorrar trabajo. No es sorprendente que si queremos etiquetar muchas imágenes usemos un "método" que no sea del todo para preciso para luego solo tener que refinar los resultados.

## Formato XML

### Usando el software de dlib o una web
https://github.com/NaturalIntelligence/imglab


### Usando las anotaciones de las que ya disponemos
En nuestro caso no tiene mucho sentido anotar otra vez las imágenes porque ya disponemos de esa información, por lo tanto lo único que tendremos que hacer es "convertir" el formato a xml para que dlib lo pueda interpretar.

In [ ]:
 """
<?xml version='1.0' encoding='ISO-8859-1'?>
<?xml-stylesheet type='text/xsl' href='image_metadata_stylesheet.xsl'?>
<dataset>
    <name>Front looking faces for training dlib::get_frontal_face_detector()</name>
    <comment></comment>
    <images>
        <image file='1\a1.jpg'>
            <box top='26' left='33' width='78' height='73'>
              <label>Clase1</label>
            </box>
        </image>
    <images>
</dataset>
"""

In [ ]:
import xml.etree.cElementTree as ET
import xml.dom.minidom as minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")


root = ET.Element("dataset")
doc = ET.SubElement(root, "name").text = "Dataset deteccion caras Curso Altran"
images = ET.SubElement(root, "images")

for imagen in anotaciones:
    nombre_de_archivo = imagen[0][0][0]
    bboxes = imagen[1][0]
    
    imagen_actual = ET.SubElement(images, "image", file=nombre_de_archivo)
    
    for bbox in bboxes:
        (x1, y1), (x2, y2) = np.array(bbox, dtype=int)
        box = ET.SubElement(imagen_actual, "box", top=str(y1), left=str(x1), width=str(x2 - x1), height=str(y2 - y1))
        box = ET.SubElement(imagen_actual, "label").text = "cara"
xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent="   ")
with open("resources/session2/face_dataset/custom_dataset.xml", "w") as f:
    f.write(xmlstr)

### Entrenando el clasificador

In [ ]:
# Now let's do the training.  The train_simple_object_detector() function has a
# bunch of options, all of which come with reasonable default values.  The next
# few lines goes over some of these options.
options = dlib.simple_object_detector_training_options()
# Since faces are left/right symmetric we can tell the trainer to train a
# symmetric detector.  This helps it get the most value out of the training
# data.
options.add_left_right_image_flips = False
# The trainer is a kind of support vector machine and therefore has the usual
# SVM C parameter.  In general, a bigger C encourages it to fit the training
# data better but might lead to overfitting.  You must find the best C value
# empirically by checking how well the trained detector works on a test set of
# images you haven't trained on.  Don't just leave the value set at 5.  Try a
# few different C values and see what works best for your data.
options.C = 5
options.epsilon = 0.001
options.upsample_limit = 0

# Tell the code how many CPU cores your computer has for the fastest training.
options.num_threads = 2
options.be_verbose = True

training_xml_path = "resources/session2/face_dataset/carles_training.xml"
testing_xml_path = "resources/session2/face_dataset/carles_testing.xml"

# This function does the actual training.  It will save the final detector to
# detector.svm.  The input is an XML file that lists the images in the training
# dataset and also contains the positions of the face boxes.  To create your
# own XML files you can use the imglab tool which can be found in the
# tools/imglab folder.  It is a simple graphical tool for labeling objects in
# images with boxes.  To see how to use it read the tools/imglab/README.txt
# file.  But for this example, we just use the training.xml file included with
# dlib.
detector = dlib.train_simple_object_detector(training_xml_path, "resources/session2/detector.svm", options)

In [ ]:
print("Training accuracy: {}".format(
    dlib.test_simple_object_detector(training_xml_path, "resources/session2/detector.svm")))
print("Testing accuracy: {}".format(
    dlib.test_simple_object_detector(testing_xml_path, "resources/session2/detector.svm")))

### Resultados

In [ ]:
detector = dlib.simple_object_detector("resources/session2/detector.svm")
win_det = dlib.image_window()
win_det.set_image(detector)
dlib.hit_enter_to_continue()


In [ ]:
final= io.imread("resources/session2/our_face_dataset/test/test.png")
detector = dlib.simple_object_detector("resources/session2/detector.svm")
    
bboxes = detector(final, 1)
print("number of rectangles found {}".format(len(bboxes))) 
for i, bbox in enumerate(bboxes):
    print(bbox)
    cv2.rectangle(final,(bbox.left(),bbox.top()),(bbox.right(),bbox.bottom()),(0,255,0),3)
        
plt.figure(figsize=(50, 50))
plt.imshow(final)

# Usando detectores ya entrenados

## Dlib HoG

In [ ]:
final= io.imread("resources/session2/our_face_dataset/test/test.png")
detector = dlib.get_frontal_face_detector()
bboxes = detector(final, 1)
print("number of rectangles found {}".format(len(bboxes))) 
for i, bbox in enumerate(bboxes):
    cv2.rectangle(final,(bbox.left(),bbox.top()),(bbox.right(),bbox.bottom()),(0,255,0),3)
        
plt.figure(figsize=(50, 50))
plt.imshow(final)

## Dlib CNN

In [ ]:
final= io.imread("resources/session2/our_face_dataset/test/test.png")
facerec = dlib.cnn_face_detection_model_v1("resources/session2/mmod_human_face_detector.dat")
bboxes = detector(final, 1)
print("number of rectangles found {}".format(len(bboxes))) 
for i, bbox in enumerate(bboxes):
    cv2.rectangle(final,(bbox.left(),bbox.top()),(bbox.right(),bbox.bottom()),(0,255,0),3)
        
plt.figure(figsize=(50, 50))
plt.imshow(final)